In [175]:
import numpy as np
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split

In [176]:
# Establish a connection to the SQLite database
conn = sqlite3.connect("laliga.sqlite")
#Read into pandas dataframe 
df = pd.read_sql_query("SELECT * FROM Matches", conn)
print(df.columns)
print(len(df))
pd.set_option('display.max_colwidth', None)
df.head(80)

Index(['season', 'division', 'matchday', 'date', 'time', 'home_team',
       'away_team', 'score'],
      dtype='object')
48780


,season,division,matchday,date,time,home_team,away_team,score
0,1928-1929,1,1,2/10/29,None,Arenas Club,Athletic Madrid,2:3
1,1928-1929,1,1,2/10/29,None,Espanyol,Real Unión,3:2
2,1928-1929,1,1,2/10/29,None,Real Madrid,Catalunya,5:0
3,1928-1929,1,1,2/10/29,None,Donostia,Athletic,1:1
4,1928-1929,1,1,2/12/29,None,Racing,Barcelona,0:2
...,...,...,...,...,...,...,...,...
75,1928-1929,1,16,6/9/29,None,Arenas Club,Espanyol,3:0
76,1928-1929,1,16,6/9/29,None,Racing,Donostia,6:1
77,1928-1929,1,16,6/9/29,None,Barcelona,Athletic,3:0
78,1928-1929,1,16,6/9/29,None,Real Unión,Real Madrid,0:2


In [177]:
from contextlib import contextmanager
import time

@contextmanager
def tiempo_ejecucion():
    inicio = time.time()
    yield
    final = time.time()
    print(f"Tiempo de ejecución: {final - inicio} segundos")

In [178]:
def process_and_filter_data(input_df):    
    # Create a copy of the df, and erase the Nones
    filtered_score_df = input_df[input_df['score'].notna()].copy()
    
    # Split the score to make it easier to read
    split_scores = filtered_score_df['score'].str.split(':', expand=True).astype(int)
    
    # Assign the split scores to new columns
    filtered_score_df['home_score'] = split_scores[0]
    filtered_score_df['away_score'] = split_scores[1]
    return filtered_score_df

In [179]:
def reult_calculator(row):
    if row['home_team'] > row['away_team']:
        return 1
    elif row['home_team'] < row['away_team']:
        return 2
    else:
        return 'X'

In [180]:
df_1=process_and_filter_data(df) 
#df_1 = df_1.drop(df_1.columns[[4,3,7]], axis=1) #Quitar tiempo, día y score
df_1['result'] = df_1.apply(reult_calculator, axis=1)
df_1

,season,division,matchday,date,time,home_team,away_team,score,home_score,away_score,result
0,1928-1929,1,1,2/10/29,None,Arenas Club,Athletic Madrid,2:3,2,3,2
1,1928-1929,1,1,2/10/29,None,Espanyol,Real Unión,3:2,3,2,2
2,1928-1929,1,1,2/10/29,None,Real Madrid,Catalunya,5:0,5,0,1
3,1928-1929,1,1,2/10/29,None,Donostia,Athletic,1:1,1,1,1
4,1928-1929,1,1,2/12/29,None,Racing,Barcelona,0:2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...
48345,2021-2022,2,3,8/28/21,10:00 PM,Ponferradina,Girona,2:1,2,1,1
48346,2021-2022,2,3,8/29/21,5:00 PM,SD Amorebieta,UD Almería,2:1,2,1,2
48347,2021-2022,2,3,8/29/21,7:30 PM,CD Lugo,Real Valladolid,0:2,0,2,2
48348,2021-2022,2,3,8/29/21,7:30 PM,Real Sociedad B,CF Fuenlabrada,0:0,0,0,1


In [181]:
# Estaría bien obtener la media ponderada del puesto en quedaron en las 5 ligas anteriores. Si no hay, poner por el máximo.
# Evaluar en función de las medias 

### Obtener set train y set test

In [182]:
X = df_1.iloc[:, :-1] 
y = df_1['result']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train

,season,division,matchday,date,time,home_team,away_team,score,home_score,away_score
596,1934-1935,1,10,2/3/35,None,Athletic,Donostia,7:0,7,0
30136,1970-1971,2,18,1/3/71,None,CD Castellón,Pontevedra,2:0,2,0
8339,1972-1973,1,32,5/6/73,None,CD Castellón,Valencia,1:3,1,3
12824,1987-1988,1,10,11/7/87,None,Real Madrid,Atlético Madrid,0:4,0,4
22894,2013-2014,1,12,11/3/13,5:00 PM,Atlético Madrid,Athletic,2:0,2,0
...,...,...,...,...,...,...,...,...,...,...
11284,1982-1983,1,20,1/15/83,None,Barcelona,Sevilla FC,1:0,1,0
44732,2014-2015,2,3,9/7/14,5:00 PM,CD Mirandés,Alcorcón,1:2,1,2
38158,1999-2000,2,35,4/16/00,None,CD Tenerife,Recr. Huelva,1:2,1,2
860,1939-1940,1,10,2/4/40,None,Athletic,Valencia,1:2,1,2


### Obtener tabla de puntuaciones obtenidas al día

In [183]:
def separate_wins_loses_ties(df_filtered):
    df_filtered['Win_local'] = np.where(df_filtered['home_score'] > df_filtered['away_score'], 1, 0)
    df_filtered['Win_away'] = np.where(df_filtered['home_score'] < df_filtered['away_score'], 1, 0)
    df_filtered['Ties'] = np.where(df_filtered['home_score'] == df_filtered['away_score'], 1, 0)
    
    # Combine home and away matches to identify all matches for each team
    df_teams = pd.concat([df_filtered.rename(columns={'home_team': 'team', 'Win_local': 'Win','Win_away':'Lose', 'home_score' : 'GF', 'away_score': 'GA',}),
                             df_filtered.rename(columns={'away_team': 'team', 'Win_away': 'Win','Win_local':'Lose', 'home_score' : 'GA', 'away_score': 'GF'})], ignore_index=True)
    return(df_teams)

In [184]:
def calculate_last_5(row, df_0):
    matchday = row['matchday']
    team_data = df_0[(df_0['season'] == row['season']) & (df_0['division'] == row['division']) & (df_0['team'] == row['team'])]
    last_5_matches_data = team_data[(team_data['matchday'] <= matchday) & (team_data['matchday'] >= matchday - 4)]
    last_5_matches = last_5_matches_data['Last'].apply(lambda x: 'W' if x == 1 else ('L' if x == -1 else 'T')).tolist()
    return last_5_matches

In [185]:
def all_info_match_day(df, season):
    print("Season: "+season)
    df_filtered = process_and_filter_data(df)   #YA ESTÁN FILTRADAS DE ANTES!!!
    df_filtered = df_filtered[df_filtered['season'] == season]
    
    df_conjunto = pd.DataFrame(columns=['season', 'division', 'team', 'GF', 'GA', 'GD', 'Win', 'Lose', 'Ties', 'Pts', 'matchday', 'last_5'])
    
    for division in df_filtered['division'].unique():
        df_0 = df_filtered.copy()
        df_0 = df_0[df_0['division'] == division]
        df_0 = separate_wins_loses_ties(df_filtered) [['season','division','team','GF','GA','Win','Lose','Ties','matchday']]

        df_0 =df_0.sort_values(by=['season','division','matchday'], ascending=[False,True,True])

        df_0['Last'] = df_0['Win']-df_0['Lose']
        grouped = df_0.groupby(['season', 'division', 'team'])
        features=['GF','GA','Win','Lose','Ties']

        for feature in features:
            df_0[feature] = grouped[feature].cumsum()


        df_0['Pts']=3*df_0['Win']+df_0['Ties']
        df_0['GD']=df_0['GF']-df_0['GA']
        df_0 =df_0.sort_values(by=['season','division','matchday','Pts'], ascending=[False,True,False,False])
        order=['season', 'division','team','GF','GA','GD','Win','Lose','Ties','Pts','matchday','Last']
        df_0=df_0[order]
        df_0 = df_0.reset_index()

        df_0 = df_0.drop(columns=['index'])
        df_0['last_5'] = df_0.apply(calculate_last_5, axis=1, args=[df_0]) 

        df_0 = df_0.drop(columns=['Last'])
        df_conjunto = pd.concat([df_conjunto, df_0], ignore_index=True)
        
    return df_conjunto

In [186]:
X_train['Win_local'] = np.where(X_train['home_score'] > X_train['away_score'], 1, 0)
X_train['Win_away'] = np.where(X_train['home_score'] < X_train['away_score'], 1, 0)
X_train['Ties'] = np.where(X_train['home_score'] == X_train['away_score'], 1, 0)
X_train

,season,division,matchday,date,time,home_team,away_team,score,home_score,away_score,Win_local,Win_away,Ties
596,1934-1935,1,10,2/3/35,None,Athletic,Donostia,7:0,7,0,1,0,0
30136,1970-1971,2,18,1/3/71,None,CD Castellón,Pontevedra,2:0,2,0,1,0,0
8339,1972-1973,1,32,5/6/73,None,CD Castellón,Valencia,1:3,1,3,0,1,0
12824,1987-1988,1,10,11/7/87,None,Real Madrid,Atlético Madrid,0:4,0,4,0,1,0
22894,2013-2014,1,12,11/3/13,5:00 PM,Atlético Madrid,Athletic,2:0,2,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,1982-1983,1,20,1/15/83,None,Barcelona,Sevilla FC,1:0,1,0,1,0,0
44732,2014-2015,2,3,9/7/14,5:00 PM,CD Mirandés,Alcorcón,1:2,1,2,0,1,0
38158,1999-2000,2,35,4/16/00,None,CD Tenerife,Recr. Huelva,1:2,1,2,0,1,0
860,1939-1940,1,10,2/4/40,None,Athletic,Valencia,1:2,1,2,0,1,0


In [187]:
X_train_home = X_train[X_train['Win_local'] == 1].rename(columns={'home_team': 'team', 'Win_local': 'Win','Win_away':'Lose', 'home_score' : 'GF', 'away_score': 'GA',})
X_train_visit = X_train[X_train['Win_away'] == 1].rename(columns={'away_team': 'team', 'Win_away': 'Win','Win_local':'Lose', 'away_score' : 'GF', 'home_score': 'GA',})

In [188]:
#X_train_home = X_train_home.groupby(['season','division','team'])[['GF', 'GA', 'Win']].sum().reset_index()
X_train_home

,season,division,matchday,date,time,team,away_team,score,GF,GA,Win,Lose,Ties
596,1934-1935,1,10,2/3/35,None,Athletic,Donostia,7:0,7,0,1,0,0
30136,1970-1971,2,18,1/3/71,None,CD Castellón,Pontevedra,2:0,2,0,1,0,0
22894,2013-2014,1,12,11/3/13,5:00 PM,Atlético Madrid,Athletic,2:0,2,0,1,0,0
1478,1943-1944,1,7,11/14/43,None,Atl. Aviación,Barcelona,2:1,2,1,1,0,0
45472,2015-2016,2,28,3/6/16,5:00 PM,Ponferradina,CD Numancia,1:0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37194,1997-1998,2,32,3/15/98,None,UE Lleida,Albacete,2:0,2,0,1,0,0
16850,1997-1998,1,16,12/13/97,None,Barcelona,Espanyol,3:1,3,1,1,0,0
6265,1964-1965,1,25,3/7/65,None,Espanyol,Dep. La Coruña,2:0,2,0,1,0,0
11284,1982-1983,1,20,1/15/83,None,Barcelona,Sevilla FC,1:0,1,0,1,0,0


In [189]:
#X_train_visit = X_train_visit.groupby(['season','division','team'])[['GF', 'GA', 'Win']].sum().reset_index()
X_train_visit

,season,division,matchday,date,time,home_team,team,score,GA,GF,Lose,Win,Ties
8339,1972-1973,1,32,5/6/73,None,CD Castellón,Valencia,1:3,1,3,0,1,0
12824,1987-1988,1,10,11/7/87,None,Real Madrid,Atlético Madrid,0:4,0,4,0,1,0
35290,1992-1993,2,35,5/29/93,None,Atl. Marbella,Racing,1:3,1,3,0,1,0
38108,1999-2000,2,31,3/19/00,None,Badajoz 1905,CP Mérida,0:3,0,3,0,1,0
22895,2013-2014,1,12,11/3/13,7:00 PM,Levante,Granada CF,0:1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37819,1999-2000,2,5,9/18/99,None,UE Lleida,Recr. Huelva,0:1,0,1,0,1,0
47191,2019-2020,2,17,11/23/19,4:00 PM,Rayo Vallecano,Real Zaragoza,0:1,0,1,0,1,0
44732,2014-2015,2,3,9/7/14,5:00 PM,CD Mirandés,Alcorcón,1:2,1,2,0,1,0
38158,1999-2000,2,35,4/16/00,None,CD Tenerife,Recr. Huelva,1:2,1,2,0,1,0


In [190]:
def correct_date(df):
        # Split the score to make it easier to read
        split_dates = df['date'].str.split('/', expand=True).astype(int)
    
        df_copy = df.copy()
    
        # Assign the split scores to new columns
        df_copy['day'] = split_dates[1]
        df_copy['month'] = split_dates[0]
        df_copy['year'] = split_dates[2]
    
        # Convert years to 4 digits format
        def correct_year(year):
            if year > 25:
                return year + 1900
            else:
                return year + 2000
    
        df_copy['year'] = df_copy['year'].apply(correct_year)
    
        # Transform date into the correct format for the command
        df_copy['date1'] = pd.to_datetime(df_copy[['year', 'month', 'day']])
        return(df_copy)

In [191]:
df_filtered = X_train

df_filtered['Win_local'] = np.where(df_filtered['home_score'] > df_filtered['away_score'], 1, 0)
df_filtered['Win_away'] = np.where(df_filtered['home_score'] < df_filtered['away_score'], 1, 0)
df_filtered['Ties'] = np.where(df_filtered['home_score'] == df_filtered['away_score'], 1, 0)

# Combine home and away matches to identify all matches for each team
df_teams = pd.concat([df_filtered.rename(columns={'home_team': 'team', 'Win_local': 'Win','Win_away':'Lose', 'home_score' : 'GF', 'away_score': 'GA',}),
                         df_filtered.rename(columns={'away_team': 'team', 'Win_away': 'Win','Win_local':'Lose', 'home_score' : 'GA', 'away_score': 'GF'})], ignore_index=True)

df_teams = correct_date(df_teams)
df_teams = df_teams.sort_values(by=['date1'])


In [192]:
def calculate_last_5(row, df_0):
    matchday = row['matchday']
    team_data = df_0[(df_0['season'] == row['season']) & (df_0['division'] == row['division']) & (df_0['team'] == row['team'])]
    last_5_matches_data = team_data[(team_data['matchday'] <= matchday) & (team_data['matchday'] >= matchday - 4)]
    last_5_matches = last_5_matches_data['Last'].apply(lambda x: 'W' if x == 1 else ('L' if x == -1 else 'T')).tolist()
    return last_5_matches

In [193]:
from datetime import datetime

# Convertir el string en un objeto de fecha
def get_5_previous_seasons(row):
    present_season = row['season']
    temporada_inicio = datetime.strptime(present_season.split('-')[0], '%Y')
    # Crear una lista para almacenar las temporadas anteriores
    temporadas_anteriores = []

    # Generar las temporadas anteriores
    for _ in range(5):
        temporada_inicio = temporada_inicio.replace(year=temporada_inicio.year - 1)
        temporada_anterior = temporada_inicio.strftime('%Y') + '-' + str(temporada_inicio.year + 1)
        temporadas_anteriores.append(temporada_anterior)

    return temporadas_anteriores

In [194]:
get_5_previous_seasons(X_train.iloc[-10]) 

['2018-2019', '2017-2018', '2016-2017', '2015-2016', '2014-2015']

In [195]:
row = X_train.iloc[-10]
row

season             2019-2020
division                   2
matchday                  17
date                11/23/19
time                 4:00 PM
home_team     Rayo Vallecano
away_team      Real Zaragoza
score                    0:1
home_score                 0
away_score                 1
Win_local                  0
Win_away                   1
Ties                       0
Name: 47191, dtype: object

### Obtener posición en anteriores 5 ligas

In [196]:
row_original = X_train.iloc[-2]
row = row_original.copy()

previous_seasons = get_5_previous_seasons(row) 
division = row['division']
home_team = row['home_team']
away_team = row['away_team']

df_filtered = X_train[X_train['season'].isin(previous_seasons)]
df_filtered = df_filtered[df_filtered['division'] == division]
df_filtered = process_and_filter_data(df_filtered)
df_0 = separate_wins_loses_ties(df_filtered) [['season','division','team','GF','GA','Win','Lose','Ties']]


df_0 = df_0.groupby(['season','team','division'])[['GF', 'GA', 'Win', 'Lose', 'Ties']].sum().reset_index()

df_0['Pts'] = 3*df_0['Win'] + df_0['Ties']
df_0['GD']= df_0['GF'] - df_0['GA']

df_0 =df_0.sort_values(by=['season','Pts'], ascending=[False,False])

df_0['Clasification'] = df_0.groupby(['season'])['Pts'].rank(ascending=False).astype(int)


for team in ['home_team', 'away_team']:
    team_name = row[team]
    results_team = df_0[df_0['team'] == team_name]
    display(results_team)
    if len(results_team.columns) == 0:
        avg_class = X_train[(X_train['division'] == division) & (X_train['season'] == row['season'])]['home_team'].nunique()
        victory_rate = 0.5
    else:
        avg_class = results_team['Clasification'].mean()
        victory_rate = results_team['Win'].sum() / (results_team['Win'].sum() + results_team['Lose'].sum()+ results_team['Ties'].sum())
    full_name_clasiff = team+'_clasiff'
    row[full_name_clasiff] = avg_class
    full_name_victory = team + '_victory_rate'
    row[full_name_victory] = victory_rate
    
row['diff_last_seasons_clasiff'] = row['home_team_clasiff']- row['away_team_clasiff']
display(row)

,season,team,division,GF,GA,Win,Lose,Ties,Pts,GD,Clasification
12,1935-1936,Athletic,1,48,29,13,5,1,40,19,1
1,1934-1935,Athletic,1,58,32,11,7,3,36,26,3


,season,team,division,GF,GA,Win,Lose,Ties,Pts,GD,Clasification
23,1935-1936,Valencia,1,26,35,4,8,3,15,-9,10
11,1934-1935,Valencia,1,25,37,5,9,2,17,-12,10


season                       1939-1940
division                             1
matchday                            10
date                            2/4/40
time                              None
home_team                     Athletic
away_team                     Valencia
score                              1:2
home_score                           1
away_score                           2
Win_local                            0
Win_away                             1
Ties                                 0
home_team_clasiff                  2.0
home_team_victory_rate             0.6
away_team_clasiff                 10.0
away_team_victory_rate        0.290323
diff_last_seasons_clasiff         -8.0
Name: 860, dtype: object

In [197]:
row = X_train.iloc[32300]
previous_seasons = get_5_previous_seasons(row) 
#= X_train[X_train['season'].isin(previous_seasons)]
df_filtered = process_and_filter_data(X_train)
#openpyxl for the excel
#If you want to see the table of the exercise, decoment the  next line
df_filtered= df_filtered[df_filtered['season'] != '2021-2022']

df_7 = separate_wins_loses_ties(df_filtered) [['season','division','team','GF','GA','Win','Lose','Ties']]
df_7

,season,division,team,GF,GA,Win,Lose,Ties
0,1934-1935,1,Athletic,7,0,1,0,0
1,1970-1971,2,CD Castellón,2,0,1,0,0
2,1972-1973,1,CD Castellón,1,3,0,1,0
3,1987-1988,1,Real Madrid,0,4,0,1,0
4,2013-2014,1,Atlético Madrid,2,0,1,0,0
...,...,...,...,...,...,...,...,...
76689,1982-1983,1,Sevilla FC,0,1,0,1,0
76690,2014-2015,2,Alcorcón,2,1,1,0,0
76691,1999-2000,2,Recr. Huelva,2,1,1,0,0
76692,1939-1940,1,Valencia,2,1,1,0,0


In [198]:
def calculate_last_5_mod(row, df_0):
    matchday = row['matchday']
    team_data = df_0[(df_0['team'] == row['team'])]
    last_5_matches_data = team_data[(team_data['matchday'] <= matchday) & (team_data['matchday'] >= matchday - 4)]
    last_5_matches = last_5_matches_data['Last'].apply(lambda x: 'W' if x == 1 else ('L' if x == -1 else 'T')).tolist()
    return last_5_matches

In [199]:
def all_info_match_day(df, season, division, matchday):
    df_filtered = process_and_filter_data(df)
    df_filtered = df_filtered[(df_filtered['season'] == season) & (df_filtered['division'] == division) & (df_filtered['matchday'] < matchday) ]
    
    df_0 = df_filtered.copy()
    df_0 = separate_wins_loses_ties(df_filtered)[['season','division','team','GF','GA','Win','Lose','Ties','matchday']]

    df_0 =df_0.sort_values(by=['matchday'], ascending=[True])

    df_0['Last'] = df_0['Win']-df_0['Lose']
    
    grouped = df_0.groupby(['team'])
    
    df_0['GF'] = grouped['GF'].cumsum()
    df_0['GA'] = grouped['GA'].cumsum()
    df_0['Win'] = grouped['Win'].cumsum()
    df_0['Lose'] = grouped['Lose'].cumsum()
    df_0['Ties'] = grouped['Ties'].cumsum()


    df_0['Pts']=3*df_0['Win']+df_0['Ties']
    df_0['GD']=df_0['GF']-df_0['GA']
    df_0 =df_0.sort_values(by=['matchday','Pts'], ascending=[False,False])
    order=['season', 'division','team','GF','GA','GD','Win','Lose','Ties','Pts','matchday','Last']
    df_0=df_0[order]
    df_0 = df_0.reset_index()

    df_0 = df_0.drop(columns=['index'])
    df_0['last_5'] = df_0.apply(calculate_last_5, axis=1, args=[df_0]) 

    df_0 = df_0.drop(columns=['Last'])

    return df_0

In [200]:
def all_info_match_day_mod(df, season, division, matchday):
    df_filtered = process_and_filter_data(df)
    df_filtered = df_filtered[(df_filtered['season'] == season) & (df_filtered['division'] == division) & (df_filtered['matchday'] < matchday) ]
    
    df_0 = df_filtered.copy()
    df_0 = separate_wins_loses_ties(df_filtered)[['season','division','team','GF','GA','Win','Lose','Ties','matchday']]

    df_0 =df_0.sort_values(by=['matchday'], ascending=[True])

    df_0['Last'] = df_0['Win']-df_0['Lose']
    grouped = df_0.groupby(['team'])

    features=['GF','GA','Win','Lose','Ties']
    
    for feature in features:
        df_0[feature] = grouped[feature].cumsum()

    df_0['Pts']=3*df_0['Win']+df_0['Ties']
    df_0['GD']=df_0['GF']-df_0['GA']
    df_0 =df_0.sort_values(by=['matchday','Pts'], ascending=[False,False])
    order=['season', 'division','team','GF','GA','GD','Win','Lose','Ties','Pts','matchday','Last']
    df_0=df_0[order]
    df_0 = df_0.reset_index()

    df_0 = df_0.drop(columns=['index'])
    df_0['last_5'] = df_0.apply(calculate_last_5, axis=1, args=[df_0]) 

    df_0 = df_0.drop(columns=['Last'])

    return df_0

In [201]:
def all_info_match_day_mod_teams(df, season, division, matchday, home_team, away_team):
    df_filtered = process_and_filter_data(df)
    df_filtered = df_filtered[(df_filtered['season'] == season) & (df_filtered['division'] == division) & (df_filtered['matchday'] < matchday) ]

    df_filtered = df_filtered[(df_filtered['home_team'] == home_team) | (df_filtered['home_team'] == away_team) | (df_filtered['away_team'] == home_team) | (df_filtered['away_team'] == away_team)]
    
    df_0 = df_filtered.copy()
    df_0 = separate_wins_loses_ties(df_filtered)[['season','division','team','GF','GA','Win','Lose','Ties','matchday']]

    df_0 =df_0.sort_values(by=['matchday'], ascending=[True])

    df_0['Last'] = df_0['Win']-df_0['Lose']
    grouped = df_0.groupby(['team'])

    features=['GF','GA','Win','Lose','Ties']
    
    for feature in features:
        df_0[feature] = grouped[feature].cumsum()

    df_0['Pts']=3*df_0['Win']+df_0['Ties']
    df_0['GD']=df_0['GF']-df_0['GA']
    df_0 =df_0.sort_values(by=['matchday','Pts'], ascending=[False,False])
    order=['season', 'division','team','GF','GA','GD','Win','Lose','Ties','Pts','matchday','Last']
    df_0=df_0[order]
    df_0 = df_0.reset_index()

    df_0 = df_0.drop(columns=['index'])
    
    df_0 = df_0[((df_0['team'] == home_team) | (df_0['team'] == away_team))  & (df_0['matchday'] == (matchday-1))]
    
    #resultado = df_0.iloc[0].apply(calculate_last_5, args=[df_0])
    
    #print('Resultado' + resultado)
    #df_0['last_5'] = df_0.apply(calculate_last_5, axis=1, args=[df_0]) 

    df_0 = df_0.drop(columns=['Last'])

    return df_0

In [202]:
from contextlib import contextmanager
import time

@contextmanager
def tiempo_ejecucion():
    inicio = time.time()
    yield
    final = time.time()
    print(f"Tiempo de ejecución: {final - inicio} segundos")

In [203]:
with tiempo_ejecucion():
    df_final = all_info_match_day_mod(X_train, "2018-2019", 1, 38)

Tiempo de ejecución: 0.4517519474029541 segundos


In [204]:
with tiempo_ejecucion():
    df_final_2 = all_info_match_day_mod_teams(X_train, "2018-2019", 1, 38, 'Atlético Madrid', 'Barcelona')

Tiempo de ejecución: 0.08227753639221191 segundos


In [205]:
df_final_2

,season,division,team,GF,GA,GD,Win,Lose,Ties,Pts,matchday
0,2018-2019,1,Atlético Madrid,46,26,20,19,6,7,64,37
1,2018-2019,1,Barcelona,64,22,42,19,2,4,61,37


In [206]:
df_final

,season,division,team,GF,GA,GD,Win,Lose,Ties,Pts,matchday,last_5
0,2018-2019,1,Atlético Madrid,46,26,20,19,6,7,64,37,"[T, L, W, W, W]"
1,2018-2019,1,Barcelona,64,22,42,19,2,4,61,37,"[W, L, W, W, W]"
2,2018-2019,1,Sevilla FC,49,38,11,13,9,7,46,37,"[T, L, L, L]"
3,2018-2019,1,Getafe,35,28,7,11,8,11,44,37,"[L, T, W]"
4,2018-2019,1,Valencia,36,26,10,10,5,12,42,37,"[W, W, L, L, W]"
...,...,...,...,...,...,...,...,...,...,...,...,...
589,2018-2019,1,SD Eibar,1,2,-1,0,1,0,0,1,[L]
590,2018-2019,1,CD Leganés,1,2,-1,0,1,0,0,1,[L]
591,2018-2019,1,Alavés,0,3,-3,0,1,0,0,1,[L]
592,2018-2019,1,Getafe,0,2,-2,0,1,0,0,1,[L]


### Get all information

In [207]:
df_all_summary = separate_wins_loses_ties(X_train) [['season','division','team','GF','GA','Win','Lose','Ties']]
df_all_summary = df_all_summary.groupby(['season','division','team'])[['GF', 'GA', 'Win', 'Lose', 'Ties']].sum().reset_index()
df_all_summary['Pts']=3*df_all_summary['Win'] + df_all_summary['Ties']
df_all_summary['GD'] = df_all_summary['GF'] - df_all_summary['GA']
df_all_summary

,season,division,team,GF,GA,Win,Lose,Ties,Pts,GD
0,1928-1929,1,Arenas Club,27,38,6,7,3,21,-11
1,1928-1929,1,Athletic,39,33,7,6,4,25,6
2,1928-1929,1,Athletic Madrid,43,38,8,7,2,26,5
3,1928-1929,1,Barcelona,34,22,10,4,2,32,12
4,1928-1929,1,Catalunya,39,46,5,8,3,18,-7
...,...,...,...,...,...,...,...,...,...,...
2781,2021-2022,2,SD Huesca,3,2,1,1,0,3,1
2782,2021-2022,2,Sporting Gijón,3,1,2,0,1,7,2
2783,2021-2022,2,UD Almería,6,4,2,1,0,6,2
2784,2021-2022,2,UD Ibiza,4,3,1,0,1,4,1


In [208]:
def split_season(df):
    df[['date1', 'date2']] = df['season'].str.split('-', expand=True)
    df['date1'] = df['date1'].astype(int)
    df['date2'] = df['date2'].astype(int)
    #df.drop('season', axis=1, inplace=True)
    return df

In [209]:
df_all_summary = split_season(df_all_summary)
df_all_summary

,season,division,team,GF,GA,Win,Lose,Ties,Pts,GD,date1,date2
0,1928-1929,1,Arenas Club,27,38,6,7,3,21,-11,1928,1929
1,1928-1929,1,Athletic,39,33,7,6,4,25,6,1928,1929
2,1928-1929,1,Athletic Madrid,43,38,8,7,2,26,5,1928,1929
3,1928-1929,1,Barcelona,34,22,10,4,2,32,12,1928,1929
4,1928-1929,1,Catalunya,39,46,5,8,3,18,-7,1928,1929
...,...,...,...,...,...,...,...,...,...,...,...,...
2781,2021-2022,2,SD Huesca,3,2,1,1,0,3,1,2021,2022
2782,2021-2022,2,Sporting Gijón,3,1,2,0,1,7,2,2021,2022
2783,2021-2022,2,UD Almería,6,4,2,1,0,6,2,2021,2022
2784,2021-2022,2,UD Ibiza,4,3,1,0,1,4,1,2021,2022


In [210]:
df_all_summary

,season,division,team,GF,GA,Win,Lose,Ties,Pts,GD,date1,date2
0,1928-1929,1,Arenas Club,27,38,6,7,3,21,-11,1928,1929
1,1928-1929,1,Athletic,39,33,7,6,4,25,6,1928,1929
2,1928-1929,1,Athletic Madrid,43,38,8,7,2,26,5,1928,1929
3,1928-1929,1,Barcelona,34,22,10,4,2,32,12,1928,1929
4,1928-1929,1,Catalunya,39,46,5,8,3,18,-7,1928,1929
...,...,...,...,...,...,...,...,...,...,...,...,...
2781,2021-2022,2,SD Huesca,3,2,1,1,0,3,1,2021,2022
2782,2021-2022,2,Sporting Gijón,3,1,2,0,1,7,2,2021,2022
2783,2021-2022,2,UD Almería,6,4,2,1,0,6,2,2021,2022
2784,2021-2022,2,UD Ibiza,4,3,1,0,1,4,1,2021,2022


### Obtener media GF de hasta 5 ligas anteriores

In [211]:
def cumsum_n(n):
        cumsum = 0
        for i in range(1, n + 1):
            cumsum += i
        return cumsum

In [212]:
def get_mean_GF(df_all_summary, team, season):
    season_format = int(season.split('-')[0])
    team_info = df_all_summary[(df_all_summary['team'] == team) & (((season_format-5) <= df_all_summary['date1'])) & (season_format > df_all_summary['date1'])].reset_index()
    tam_df = team_info.shape[0]
    if tam_df == 0:
        mean_GF_team = df_all_summary['GF'].mean() * 0.6
    else:
        team_info['GF_pondered'] = team_info['GF'] * (team_info.index+1)
        mean_GF_team = team_info['GF_pondered'].sum()/(cumsum_n(tam_df))
    #display(team_info)  ## Se puede comentar
    return mean_GF_team 

In [213]:
def get_mean_parameter(df_all_summary, team, season, parameter, division):
    season_format = int(season.split('-')[0])
    team_info = df_all_summary[(df_all_summary['team'] == team) & (((season_format-5) <= df_all_summary['date1'])) & (season_format > df_all_summary['date1'])].reset_index()
    tam_df = team_info.shape[0]
    if tam_df == 0:
        mean_parameter_team = df_all_summary[parameter].mean() #* 0.6
    else:
        team_info['parameter_pondered'] = team_info[parameter] * (team_info.index+1)
        mean_parameter_team  = team_info['parameter_pondered'].sum()/(cumsum_n(tam_df))
    if ((division == 2) & (parameter == 'Pts')):
        mean_parameter_team = mean_parameter_team - df_all_summary[parameter].mean()*2
    #display(team_info)  ## Se puede comentar
    return mean_parameter_team 

In [214]:
def get_teams_last_mean_dif(row_original, df_all_summary):
    row = row_original.copy()
    home_team = row['home_team']  
    away_team = row['away_team']
    division_team = row['division']
    season = row['season']  
    row['GF_mean_home'] = get_mean_GF(df_all_summary, home_team, season)
    row['GF_mean_away'] = get_mean_GF(df_all_summary, away_team, season)
    row['dif_GF_mean'] = row['GF_mean_home'] - row['GF_mean_away']
    return row

In [215]:
def get_teams_last_mean_dif_p(row_original, df_all_summary, parameter):
    row = row_original.copy()
    home_team = row['home_team']  
    away_team = row['away_team']  
    season = row['season']  
    row[parameter+'_mean_home'] = get_mean_parameter(df_all_summary, home_team, season, parameter, division)
    row[parameter+'_mean_away'] = get_mean_parameter(df_all_summary, away_team, season, parameter, division)
    row['dif_'+parameter+'_mean'] = row[parameter+'_mean_home'] - row[parameter+'_mean_away']
    return row

In [216]:
row_original = X_train.iloc[120]
get_teams_last_mean_dif(row_original, df_all_summary)

season            1999-2000
division                  2
matchday                 39
date                5/14/00
time                   None
home_team       CD Tenerife
away_team        CD Leganés
score                   0:0
home_score                0
away_score                0
Win_local                 0
Win_away                  0
Ties                      1
GF_mean_home           43.0
GF_mean_away           29.0
dif_GF_mean            14.0
Name: 38202, dtype: object

In [217]:
row_2 = get_teams_last_mean_dif_p(row_original, df_all_summary, 'GD')
row_2 = get_teams_last_mean_dif_p(row_2, df_all_summary, 'Pts')
row_2

season             1999-2000
division                   2
matchday                  39
date                 5/14/00
time                    None
home_team        CD Tenerife
away_team         CD Leganés
score                    0:0
home_score                 0
away_score                 0
Win_local                  0
Win_away                   0
Ties                       1
GD_mean_home            -3.0
GD_mean_away       -3.066667
dif_GD_mean         0.066667
Pts_mean_home      39.733333
Pts_mean_away           37.4
dif_Pts_mean        2.333333
Name: 38202, dtype: object

In [218]:
row_original = X_train.iloc[1]
row = row_original.copy()
home_team = row['home_team']  
away_team = row['away_team']  
season = row['season']  
row['GF_mean_home'] = get_mean_GF(df_all_summary, home_team, season)
row['GF_mean_away'] = get_mean_GF(df_all_summary, away_team, season)
row['dif_GF_mean'] = row['GF_mean_home'] - row['GF_mean_away']

In [219]:
row

season             1970-1971
division                   2
matchday                  18
date                  1/3/71
time                    None
home_team       CD Castellón
away_team         Pontevedra
score                    2:0
home_score                 2
away_score                 0
Win_local                  1
Win_away                   0
Ties                       0
GF_mean_home        22.16705
GF_mean_away            19.4
dif_GF_mean          2.76705
Name: 30136, dtype: object

### Transformar últimas victorias en puntos

In [220]:
df_filtered = process_and_filter_data(X_train)


df_0 = separate_wins_loses_ties(df_filtered) [['season','division','team','GF','GA','Win','Lose','Ties','matchday']]
df_0 =df_0.sort_values(by=['season','division','matchday'], ascending=[False,True,True])

df_0['Last'] = df_0['Win']-df_0['Lose']
grouped = df_0.groupby(['season', 'division', 'team'])
df_0['GF'] = grouped['GF'].cumsum()
df_0['GA'] = grouped['GA'].cumsum()
df_0['Win'] = grouped['Win'].cumsum()
df_0['Lose'] = grouped['Lose'].cumsum()
df_0['Ties'] = grouped['Ties'].cumsum()


df_0['Pts']=3*df_0['Win']+df_0['Ties']
df_0['GD']=df_0['GF']-df_0['GA']
df_0 =df_0.sort_values(by=['season','division','matchday','Pts'], ascending=[False,True,False,False])
order=['season', 'division','team','GF','GA','GD','Win','Lose','Ties','Pts','matchday','Last']
df_0=df_0[order]
df_0 = df_0.reset_index()

df_0 = df_0.drop(columns=['index'])


for i in range(5):
   df_0[f"last_{i}"] = df_0.groupby(['division','season' ,'team'])['Last'].shift(-i)




df_0["last_5"] = df_0[[f"last_{i}" for i in range(5)]].agg(lambda x: [i for i in x if not pd.isna(i)],axis=1)

df_0 = df_0.drop(columns=['Last'])
for i in range(5):
    df_0 = df_0.drop(columns=[f"last_{i}"])

df_0['Sum'] = df_0['last_5'].apply(sum)

df_0['GD']= df_0.groupby(['division','season' ,'team'])['GD'].shift(-1)
df_0['Pts']= df_0.groupby(['division','season' ,'team'])['Pts'].shift(-1)
df_0['Sum']= df_0.groupby(['division','season' ,'team'])['Sum'].shift(-1)
df_0['N'] = df_0['matchday'].apply(lambda x: 1 if x == 0 else 1)
df_0['GD'] = df_0['N'] * df_0['GD']
df_0['Pts'] = df_0['N'] * df_0['Pts']
df_0['Sum'] = df_0['N'] * df_0['Sum']
df_0.fillna(0, inplace=True)

display(df_0)

,season,division,team,GF,GA,GD,Win,Lose,Ties,Pts,matchday,last_5,Sum,N
0,2021-2022,1,RCD Mallorca,3,1,1.0,2,0,1,4.0,3,"[1.0, 1.0, 0.0]",1.0,1
1,2021-2022,1,Barcelona,7,4,2.0,2,0,1,4.0,3,"[1.0, 0.0, 1.0]",1.0,1
2,2021-2022,1,Real Madrid,8,4,3.0,2,0,1,4.0,3,"[1.0, 0.0, 1.0]",1.0,1
3,2021-2022,1,Sevilla FC,5,1,4.0,2,0,1,6.0,3,"[0.0, 1.0, 1.0]",2.0,1
4,2021-2022,1,Athletic,2,1,0.0,1,0,2,2.0,3,"[1.0, 0.0, 0.0]",0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76795,1928-1929,1,Athletic Madrid,3,2,0.0,1,0,0,0.0,1,[1.0],0.0,1
76796,1928-1929,1,Donostia,1,1,0.0,0,0,1,0.0,1,[0.0],0.0,1
76797,1928-1929,1,Athletic,1,1,0.0,0,0,1,0.0,1,[0.0],0.0,1
76798,1928-1929,1,Arenas Club,2,3,0.0,0,1,0,0.0,1,[-1.0],0.0,1


In [221]:
df_0=df_0[['season','matchday','team','Pts','Sum','GD']]
def get_matchday_info(row_original, df_0):
    row = row_original
    row = row_original.copy()
    home_team = row['home_team']  
    away_team = row['away_team']  
    season = row['season']  
    matchday = row['matchday']
    df_0_h= df_0[(df_0['season'] == season) & (df_0['matchday'] == matchday) & (df_0['team'] == home_team)]
    df_0_a=df_0[(df_0['season'] == season) & (df_0['matchday'] == matchday) & (df_0['team'] == away_team)]


    row['yet_pts_dif'] = df_0_h['Pts'].values[0] - df_0_a['Pts'].values[0]
    
    row['yet_sum5_dif'] = df_0_h['Sum'].values[0] -df_0_a['Sum'].values[0]
    
    row['yet_GD_dif'] = df_0_h['GD'].values[0] - df_0_a['GD'].values[0]
    
    
    return row

In [222]:
row_2 = get_matchday_info(row_2, df_0)
row_2


season             1999-2000
division                   2
matchday                  39
date                 5/14/00
time                    None
home_team        CD Tenerife
away_team         CD Leganés
score                    0:0
home_score                 0
away_score                 0
Win_local                  0
Win_away                   0
Ties                       1
GD_mean_home            -3.0
GD_mean_away       -3.066667
dif_GD_mean         0.066667
Pts_mean_home      39.733333
Pts_mean_away           37.4
dif_Pts_mean        2.333333
yet_pts_dif             -9.0
yet_sum5_dif            -4.0
yet_GD_dif              12.0
Name: 38202, dtype: object

In [223]:
final_columns = ['dif_GD_mean','dif_Pts_mean','yet_GD_dif','yet_pts_dif','yet_sum5_dif']
X_process = X_train[['season','division','matchday','home_team','away_team']]
X_process=X_process.head(10000)

In [224]:

def get_matchday_info(row_original, df_0):
    row = row_original.copy()
    home_team = row['home_team']
    away_team = row['away_team']
    season = row['season']
    matchday = row['matchday']

    
    df_0_matchday = df_0[(df_0['season'] == season) & (df_0['matchday'] == matchday)]
    
    home_info = df_0_matchday[df_0_matchday['team'] == home_team].iloc[0]
    away_info = df_0_matchday[df_0_matchday['team'] == away_team].iloc[0]

    row['yet_pts_dif'] = home_info['Pts'] - away_info['Pts']
    row['yet_sum5_dif'] = home_info['Sum'] - away_info['Sum']
    row['yet_GD_dif'] = home_info['GD'] - away_info['GD']

    return row

X_process1 = X_process.apply(lambda row: get_matchday_info(row, df_0), axis=1)



In [225]:
df_0.set_index(['season', 'matchday', 'team'], inplace=True)

def get_matchday_info(row_original, df_0):
    row = row_original.copy()
    home_team = row['home_team']
    away_team = row['away_team']
    season = row['season']
    matchday = row['matchday']

    # Busca los datos de los equipos local y visitante directamente por su índice
    home_info = df_0.loc[(season, matchday, home_team)]
    away_info = df_0.loc[(season, matchday, away_team)]

    row['yet_pts_dif'] = home_info['Pts'] - away_info['Pts']
    row['yet_sum5_dif'] = home_info['Sum'] - away_info['Sum']
    row['yet_GD_dif'] = home_info['GD'] - away_info['GD']

    return row

X_process1 = X_process.apply(lambda row: get_matchday_info(row, df_0), axis=1)


/tmp/ipykernel_1217/42797736.py:11: PerformanceWarning: indexing past lexsort depth may impact performance.
  home_info = df_0.loc[(season, matchday, home_team)]
/tmp/ipykernel_1217/42797736.py:12: PerformanceWarning: indexing past lexsort depth may impact performance.
  away_info = df_0.loc[(season, matchday, away_team)]
/tmp/ipykernel_1217/42797736.py:11: PerformanceWarning: indexing past lexsort depth may impact performance.
  home_info = df_0.loc[(season, matchday, home_team)]
/tmp/ipykernel_1217/42797736.py:12: PerformanceWarning: indexing past lexsort depth may impact performance.
  away_info = df_0.loc[(season, matchday, away_team)]
/tmp/ipykernel_1217/42797736.py:11: PerformanceWarning: indexing past lexsort depth may impact performance.
  home_info = df_0.loc[(season, matchday, home_team)]
/tmp/ipykernel_1217/42797736.py:12: PerformanceWarning: indexing past lexsort depth may impact performance.
  away_info = df_0.loc[(season, matchday, away_team)]
/tmp/ipykernel_1217/42797736

In [227]:
df_0=df_0[['season','matchday','team','Pts','Sum','GD']]
def get_matchday_info(row_original, df_0):
    row = row_original
    row = row_original.copy()
    home_team = row['home_team']  
    away_team = row['away_team']  
    season = row['season']  
    matchday = row['matchday']
    df_0_h= df_0[(df_0['season'] == season) & (df_0['matchday'] == matchday) & (df_0['team'] == home_team)]
    df_0_a=df_0[(df_0['season'] == season) & (df_0['matchday'] == matchday) & (df_0['team'] == away_team)]


    row['yet_pts_dif'] = df_0_h['Pts'].values[0] - df_0_a['Pts'].values[0]
    
    row['yet_sum5_dif'] = df_0_h['Sum'].values[0] -df_0_a['Sum'].values[0]
    
    row['yet_GD_dif'] = df_0_h['GD'].values[0] - df_0_a['GD'].values[0]
    
    
    return row
clean_X_train = X_train[['season','division','matchday','home_team','away_team']]

X_process1 = X_process.apply(lambda row: get_matchday_info(row, df_0), axis=1)
print("Segundo terminado")

KeyError: "['season', 'matchday', 'team'] not in index"

In [ ]:
X_process1=X_process

: 

In [ ]:
X_process2=X_process2.apply(lambda row: get_teams_last_mean_dif_p(row, df_all_summary, 'Pts'), axis=1)

: 

In [ ]:

display(X_process2)
X_process3 = X_process2[['dif_GF_mean','dif_Pts_mean','yet_GD_dif','yet_pts_dif','yet_sum5_dif']]
X_process3
X_process2.to_csv('X_train.txt', sep='\t', index=False)

: 

In [ ]:
clean_X_test = X_test[['season','division','matchday','home_team','away_team']]

X_test1 = clean_X_test.apply(lambda row: get_teams_last_mean_dif(row, df_all_summary), axis=1)
print("Primero terminado")

X_test1 = X_test1.apply(lambda row: get_matchday_info(row, df_0), axis=1)
print("Segundo terminado")
X_test1=X_test1.apply(lambda row: get_teams_last_mean_dif_p(row, df_all_summary, 'Pts'), axis=1)

X_test1.to_csv('X_test.txt', sep='\t', index=False)

: 

In [ ]:
from sklearn.naive_bayes import GaussianNB
clf_NB = GaussianNB()
clf_NB.fit(X_process3, y_train)

: 